<a href="https://colab.research.google.com/github/Arbabyounis46/Project_Breast_Cancer_Classification/blob/main/Breast_Cancer_Classification_Using_CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Importing the Drive to Use Dataset

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Importing The Libraries

In [ ]:
# Importing necessary libraries
import os  # For file and directory operations
import numpy as np  # For numerical operations
import pandas as pd  # For data manipulation and analysis

import tensorflow as tf  # For building and training deep learning models

from sklearn.metrics import *  # For evaluating model performance

import matplotlib.pyplot as plt  # For data visualization
import seaborn as sns  # For enhanced data visualization

from tensorflow.keras.preprocessing.image import ImageDataGenerator  # For image augmentation
from tensorflow.keras import layers, models  # For building deep learning models

# Enable data loading in parallel to improve performance
AUTOTUNE = tf.data.experimental.AUTOTUNE

# Load the CSV file containing information about the data folds
folds_data = pd.read_csv('/content/drive/MyDrive/Data Set/Folds.csv')

# Specify the path where images are stored
images_path = '/content/drive/MyDrive/Data Set/BreaKHis_v1'

# Define tumor classes for classification
tumor_classes = ['benign', 'malignant']

# The code below this would likely involve data preprocessing, model building, and training.


# Removing the Duplicate Values in the CSV File

In [ ]:
# Check for duplicate filenames
duplicate_filename_count = folds_data['filename'].duplicated().sum()
print(f'Number of duplicate filenames: {duplicate_filename_count}')

# Remove duplicate rows based on the 'filename' column to retain only unique entries
folds_data = folds_data.drop_duplicates(subset='filename', keep='first')

# Print the updated shape of the DataFrame
print(f'Updated shape of the DataFrame: {folds_data.shape}')


In [ ]:
# Rename the 'filename' column to 'path'
folds_data = folds_data.rename(columns={'filename': 'path'})

# Extract and assign labels from the path (assuming the label is the 4th part of the path)
folds_data['label'] = folds_data['path'].apply(lambda x: x.split('/')[3])

# Convert labels to integer indices based on the defined tumor_classes list
folds_data['label_int'] = folds_data['label'].apply(lambda x: tumor_classes.index(x))

# Extract and assign file names from the path
folds_data['file_name'] = folds_data['path'].apply(lambda x: x.split('/')[-1])

# Update the 'path' column to include the full image folder path
folds_data['path'] = folds_data['path'].apply(lambda x: images_path + x)

# Display the first three rows of the DataFrame to verify the changes
folds_data.head(3)


In [1]:
# Defining colors for tumor classes
ax = sns.displot(
    data=folds_data,
    x='label',
    hue='label',
    palette={'benign': 'lightgreen', 'malignant': 'lightpink'}  # Changed colors
)

# Displaying the count of each label
print('Count of Benign    : ', folds_data[folds_data['label'] == 'benign']['label'].count())
print('Count of Malignant : ', folds_data[folds_data['label'] == 'malignant']['label'].count())


NameError: name 'sns' is not defined

In [ ]:
# Remove 600 samples from the dataset for testing (300 per class)
test_data = folds_data.groupby('label').sample(n=300)
train_data = folds_data.drop(test_data.index).reset_index(drop=True)
test_data = test_data.reset_index(drop=True)

# Split training and validation sets
validation_data = train_data.sample(frac=0.2)
train_data = train_data.drop(validation_data.index).reset_index(drop=True)
validation_data = validation_data.reset_index(drop=True)

# Assign set labels and combine data
test_data['set'] = 'test'
train_data['set'] = 'train'
validation_data['set'] = 'valid'
combined_data = pd.concat([train_data, validation_data, test_data])

# Set up the figure with a column of three subplots
fig, axes = plt.subplots(3, 1, figsize=(8, 15))

# Plot each distribution with different colors and assign to specific axes
sns.histplot(data=train_data, x='label', color='teal', ax=axes[0])
axes[0].set_title('Training Set')

sns.histplot(data=validation_data, x='label', color='purple', ax=axes[1])
axes[1].set_title('Validation Set')

sns.histplot(data=test_data, x='label', color='orange', ax=axes[2])
axes[2].set_title('Test Set')

# Show the plots in a vertical layout
plt.tight_layout()
plt.show()

# Print counts for each set
print('Training set')
print(train_data['label'].value_counts())

print('\nValidation set')
print(validation_data['label'].value_counts())

print('\nTest set')
print(test_data['label'].value_counts())


In [ ]:
# Upsample the training dataset to balance the classes
max_count = np.max(train_data['label'].value_counts())
train_data = train_data.groupby('label').sample(n=max_count, replace=True).reset_index(drop=True)

# Plot distribution after upsampling
ax = sns.displot(data=train_data, x='label', color='mediumslateblue')
ax.set(title='Upsampled Training Set')
plt.show()

# Display counts to verify upsampling
print(train_data['label'].value_counts())


In [ ]:
# Convert 'label_int' to string type for categorical class mode
train_data['label_int'] = train_data['label_int'].astype(str)
validation_data['label_int'] = validation_data['label_int'].astype(str)

# Data augmentation setup with fewer parameters for simplicity
train_image_gen = ImageDataGenerator(
    rescale=1./255,  # Normalize pixel values to [0, 1]
    rotation_range=10,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

# Use a simpler validation generator with rescaling only
validation_image_gen = ImageDataGenerator(rescale=1./255)

# Training data generator setup
train_data_gen = train_image_gen.flow_from_dataframe(
    dataframe=train_data,
    directory='',
    x_col='path',
    y_col='label_int',
    target_size=(210, 210),
    batch_size=64,
    class_mode='sparse'
)

# Validation data generator setup
validation_data_gen = validation_image_gen.flow_from_dataframe(
    dataframe=validation_data,
    directory='',
    x_col='path',
    y_col='label_int',
    target_size=(210, 210),
    batch_size=64,
    class_mode='sparse'
)


# Defining the Model Architecture

In [ ]:
model = models.Sequential()

# First convolutional block
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(210, 210, 3)))
model.add(layers.BatchNormalization())
model.add(layers.Conv2D(32, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Dropout(0.2))

# Second convolutional block
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.BatchNormalization())
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Dropout(0.25))

# Third convolutional block
model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.BatchNormalization())
model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Dropout(0.3))

# Fourth convolutional block
model.add(layers.Conv2D(256, (3, 3), activation='relu'))
model.add(layers.BatchNormalization())
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Dropout(0.35))

# Flatten and fully connected layers
model.add(layers.Flatten())
model.add(layers.Dense(256, activation='relu'))
model.add(layers.BatchNormalization())
model.add(layers.Dropout(0.4))
model.add(layers.Dense(128, activation='relu'))
model.add(layers.BatchNormalization())
model.add(layers.Dropout(0.4))
model.add(layers.Dense(2, activation='softmax'))  # Output layer for 2 classes

# Print model summary to verify structure
model.summary()


In [ ]:
# Compile the model with Adam optimizer, sparse categorical cross-entropy loss, and accuracy metric
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# Display the model summary to verify the final architecture
model.summary()


In [ ]:
# Train the model
history = model.fit(
    train_data_generator,        # Training data generator
    epochs=20,                   # Number of epochs (adjust as needed)
    validation_data=validation_data_generator,  # Validation data generator
    steps_per_epoch=len(train_data_generator),  # Number of steps per epoch
    validation_steps=len(validation_data_generator),  # Number of validation steps per epoch
    verbose=1                    # Verbosity mode for training progress output
)

# Save the trained model if needed
model.save('custom_cnn_model_breast_cancer.h5')
